In [ ]:
!guardrails hub install hub://brainlogic/high_quality_translation

# Translate text with quality checks

**Note:**
To download this example as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/translation_with_quality_check.ipynb).

In this example, we will use Guardrails during the translation of a statement from another language to English. We will check whether the translated statement is likely of high quality.

## Objective

We want to translate a statement from different languages to English and ensure that the translated statement accurately reflects the original content.

### Setup

- Install the `unbabel-comet` from source:
  `pip install git+https://github.com/Unbabel/COMET`
- Please accept the model license from:
  https://huggingface.co/Unbabel/wmt22-cometkiwi-da
- Login into Huggingface Hub using:
  huggingface-cli login --token $HUGGINGFACE_TOKEN


In [ ]:
from pydantic import BaseModel, Field
from guardrails import Guard
from rich import print
from guardrails.hub import HighQualityTranslation

## Step 1: Define a Guard that uses the validator

This guard will use the HighQualityTranslation validator to validate some string outputs.


We define the prompt and the guard.

In [ ]:
prompt = """
Translate the given statement into English:

${statement_to_be_translated}
"""

guard = Guard().use(HighQualityTranslation(on_fail="fix"))


## Step 2: Wrap the LLM API call with `Guard`


First, let's try translating a statement that is relatively easy to translate.


In [ ]:
# Set your OPENAI_API_KEY as an environment variable
# import os
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

statement = "Ich habe keine Ahnung, was ich hier schreiben soll."

res = guard(
    messages=[{"role":"user", "content": prompt}],
    prompt_params={"statement_to_be_translated": statement},
    metadata={"translation_source": statement},
    model="gpt-3.5-turbo",
    max_tokens=1024,
    temperature=0,
)

print(f"Raw LLM Output: {res.raw_llm_output}")
print(f"Validated Output: {res.validated_output}")

We can look at the logs to see the quality check results:


In [ ]:
print(guard.history.last.tree)

The `guard` wrapper returns the raw LLM response, which is the translated statement and also the validated output. In this case, the translated statement was of a good quality (above the threshold of 0.5), so the validated output is the same as the raw LLM response.

#### Now, let's test with a really low quality translation, and see how Guardrails handles it.


In [ ]:
# Parse the code snippet
statement = "अरे भाऊ, आज रात्री जोरदार पार्टी मारूया, जमून टाकूया आणि धमाल करूया!"

## Ideal translation from Marathi -> English:
#  "Hey bro, let's have a great party tonight and have fun!"

output = guard.parse(
    llm_output="It's such a beautiful day, I'm going to the beach.",  ## here, providing a really bad translation
    metadata={"translation_source": statement},
)

# Print the output
print(f"Raw LLM Output: {output.raw_llm_output}")
print(f"Validated Output: {output.validated_output}")

In [ ]:
print(guard.history.last.tree)

As you can see, the translation quality is really bad, and the `HighQualityTranslation` check failed as the translation quality was below the threshold. The validated response is an empty string.

## In this way, you can use Guardrails to ensure that the output of your LLM is of high quality.
